## Импорт зависимостей

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Вспомогательные функции

In [2]:
def get_hex(number: float | int):
    if type(number) == int:
        return f'{number:x}'
    point_position: int = str(number).find(".")
    return f'{int(str(number)[0:point_position]):x}.{int(str(number)[point_position + 1:]):x}'

## Инициализация целевой функции

In [3]:
# Определение целевой функции
class TargetFunction:
    def __init__(
            self, 
            name: str, 
            surname: str
    ) -> None:
        self.i: float = len(name) / 10
        self.f: float = len(surname) / 10
        self.calculations_count: int = 0
        self.derivative_calculations_count: int = 0

    def calculate(
            self, 
            x: np.array, 
            add_calculations_count: bool = False
    ) -> float:
        if x.shape[0] != 2:
            raise ValueError('x not from R^2')
        if add_calculations_count:
            self.calculations_count += 1
        return np.cosh(self.i * x[0]) + np.cosh(self.f * x[1]) + x[0] + x[1]

    def get_calculations_count(self) -> int:
        return self.calculations_count

    def flush_calculations_count(self) -> None:
        self.calculations_count = 0
        print('Calculations count flush - Success')

    def calculate_partial_derivative(
            self, 
            x: np.array, 
            variable_index: int, 
            add_calculations_count: bool = False
    ) -> float:
        if x.shape[0] != 2:
            raise ValueError('x not from R^2')
        if add_calculations_count:
            self.derivative_calculations_count += 1
        if variable_index == 0:
            return np.sinh(self.i * x[0]) + 1
        elif variable_index == 1:
            return np.sinh(self.f * x[1]) + 1
        raise ValueError('Wrong variable')

    def get_derivative_calculations_count(self) -> int:
        return self.derivative_calculations_count

    def flush_derivative_calculations_count(self) -> str:
        self.derivative_calculations_count = 0
        print('Derivative calculations count flush - Success')

    def calculate_gradient(
            self, 
            x: np.array,
            add_calculations_count: bool = False
    ) -> np.array:
        if x.shape[0] != 2:
            raise ValueError('x not from R^2')
        return np.array([
            self.calculate_partial_derivative(x, 0, add_calculations_count), 
            self.calculate_partial_derivative(x, 1, add_calculations_count)
        ])

## Поиск экстремума методом внешних штрафов

In [4]:
# Целевая функция с внешними штрафами
class ConditionalTargetFunctionWithExternalPenalties(TargetFunction):
    def __init__(
            self,
            name: str,
            surname: str,
            r: float,
            alpha: float,
            penalty_coefficient: float = 0.1
    ):
        super().__init__(name, surname)
        self.penalty_coefficient = penalty_coefficient
        self.r = r
        self.alpha = alpha
        
    def calculate(
            self, 
            x: np.array, 
            add_calculations_count: bool = False
    ) -> float:
        if x.shape[0] != 2:
            raise ValueError('x not from R^2')
        return super().calculate(x, add_calculations_count) + self.penalty_coefficient * np.max([0, x[0] ** 2 + x[1] ** 2 - self.r ** 2]) ** 2 + self.penalty_coefficient * np.max([0, x[0] + self.alpha * x[1]]) ** 2
    
    def calculate_partial_derivative(
            self, 
            x: np.array, 
            variable_index: int, 
            add_calculations_count: bool = False
    ) -> float:
        if x.shape[0] != 2:
            raise ValueError('x not from R^2')
        if variable_index == 0:
            return super().calculate_partial_derivative(x, variable_index, add_calculations_count) + self.penalty_coefficient * 2 * np.max([0, x[0] ** 2 + x[1] ** 2 - self.r ** 2]) * 2 * x[0]
        elif variable_index == 1:
            return super().calculate_partial_derivative(x, variable_index, add_calculations_count) + self.penalty_coefficient * 2 * np.max([0, x[0] + self.alpha * x[1]]) * self.alpha
        raise ValueError('Wrong variable')

### Метод градиентного спуска

In [46]:
def gradient_descent(
        func: TargetFunction, 
        x_0: np.array, 
        step_size: float = 0.01, 
        stop_factor: float = 0.001, 
        max_iterations: int = 1000
) -> (np.array, float):
    for iteration in range(1, max_iterations + 1):
        gradient = func.calculate_gradient(x_0)
        x_1 =  x_0  - step_size * gradient

        # history = {
        #     'x_0': x_0,
        #     'x_1': x_1,
        #     'function_value': func.calculate(x_1),
        #     'gradient_norm': np.linalg.norm(gradient)
        # }
        # print(iteration, history)
        
        if np.linalg.norm(gradient) < stop_factor:
            return x_1, func.calculate(x_1)
        else:
            x_0 = x_1

    return x_1, func.calculate(x_1, add_calculations_count=False)

In [47]:
my_func = ConditionalTargetFunctionWithExternalPenalties('Gregory', 'Matsnev', r=1, alpha=10, penalty_coefficient=0.1)
real_func = TargetFunction('Gregory', 'Matsnev')

min_x = np.array([-0.0, -0.0])

for i in range(5):
    min_x, min_value = gradient_descent(my_func, min_x, step_size=0.01)
    my_func.penalty_coefficient *= 2

    g1 = min_x[0] ** 2 + min_x[1] ** 2 - my_func.r
    g2 = min_x[0] + my_func.alpha * min_x[1]
    
    print('\n№', i, ', c:', my_func.penalty_coefficient, f', min_x: {min_x}', f', f(min_x): {real_func.calculate(min_x)}', ', g1:', g1, ', g2:', g2)


№ 0 , c: 0.2 , min_x: [-0.81063591 -1.25838261] , f(min_x): 0.5100530988194862 , g1: 1.2406573546149064 , g2: -13.394461967829914

№ 1 , c: 0.4 , min_x: [-0.65269539 -1.25904852] , f(min_x): 0.6086310840299636 , g1: 1.011214457741993 , g2: -13.24318063159374

№ 2 , c: 0.8 , min_x: [-0.49143023 -1.25909812] , f(min_x): 0.7234344978164093 , g1: 0.8268317439355093 , g2: -13.082411414396663

№ 3 , c: 1.6 , min_x: [-0.33947633 -1.25910378] , f(min_x): 0.8440004554069143 , g1: 0.7005865073704136 , g2: -12.930514135101376

№ 4 , c: 3.2 , min_x: [-0.21139858 -1.25910471] , f(min_x): 0.9546788713623902 , g1: 0.6300340369814987 , g2: -12.80244570817166


## Оптимизация функции с внутренними штрафами

In [22]:
# Целевая функция с внутренними штрафами
class ConditionalTargetFunctionWithInternalPenalties(TargetFunction):
    def __init__(
            self,
            name: str,
            surname: str,
            r: float,
            alpha: float,
            penalty_coefficient: float = 0.1
    ):
        super().__init__(name, surname)
        self.penalty_coefficient = penalty_coefficient
        self.r = r
        self.alpha = alpha
        
    def calculate(
            self, 
            x: np.array, 
            add_calculations_count: bool = False
    ) -> float:
        if x.shape[0] != 2:
            raise ValueError('x not from R^2')
        return super().calculate(x, add_calculations_count) + self.penalty_coefficient / (x[0] ** 2 + x[1] ** 2 - self.r ** 2) + self.penalty_coefficient / (x[0] + self.alpha * x[1])
    
    def calculate_partial_derivative(
            self, 
            x: np.array, 
            variable_index: int, 
            add_calculations_count: bool = False
    ) -> float:
        if x.shape[0] != 2:
            raise ValueError('x not from R^2')
        if variable_index == 0:
            return super().calculate_partial_derivative(x, variable_index, add_calculations_count) - self.penalty_coefficient * 2 * x[0] / (x[0] ** 2 + x[1] ** 2 - self.r ** 2) ** 2 - self.penalty_coefficient / (x[0] + self.alpha * x[1]) ** 2
        elif variable_index == 1:
            return super().calculate_partial_derivative(x, variable_index, add_calculations_count) - self.penalty_coefficient * 2 * x[1] / (x[0] ** 2 + x[1] ** 2 - self.r ** 2) ** 2 - self.penalty_coefficient * self.alpha / (x[0] + self.alpha * x[1]) ** 2
        raise ValueError('Wrong variable')

### Метод градиентного спуска

In [52]:
my_func = ConditionalTargetFunctionWithExternalPenalties('Gregory', 'Matsnev', r=1, alpha=10, penalty_coefficient=-0.1)
real_func = TargetFunction('Gregory', 'Matsnev')

min_x = np.array([0.0, 0.0])

for i in range(8):
    min_x, min_value = gradient_descent(my_func, min_x, step_size=0.0001)
    my_func.penalty_coefficient /= 5

    g1 = min_x[0] ** 2 + min_x[1] ** 2 - my_func.r
    g2 = min_x[0] + my_func.alpha * min_x[1]
    
    print('\n№', i, ', c:', my_func.penalty_coefficient, f', min_x: {min_x}', f', f(min_x): {real_func.calculate(min_x)}', ', g1:', g1, ', g2:', g2)


№ 0 , c: -0.02 , min_x: [-0.09658223 -0.09658223] , f(min_x): 1.8114080698896697 , g1: -0.9813437469770914 , g2: -1.0624044935362285

№ 1 , c: -0.004 , min_x: [-0.18661836 -0.18661836] , f(min_x): 1.6438525082936573 , g1: -0.930347177384532 , g2: -2.052801931077573

№ 2 , c: -0.0008 , min_x: [-0.27051583 -0.27051583] , f(min_x): 1.494933240019387 , g1: -0.8536423748524643 , g2: -2.9756740952977214

№ 3 , c: -0.00016 , min_x: [-0.34864194 -0.34864194] , f(min_x): 1.3625724098547365 , g1: -0.7568975921207365 , g2: -3.8350613654406427

№ 4 , c: -3.2000000000000005e-05 , min_x: [-0.42133322 -0.42133322] , f(min_x): 1.244951550378313 , g1: -0.6449566283949921 , g2: -4.634665466039914

№ 5 , c: -6.400000000000001e-06 , min_x: [-0.48890231 -0.48890231] , f(min_x): 1.1404654780204968 , g1: -0.5219490709433958 , g2: -5.377925362807162

№ 6 , c: -1.2800000000000002e-06 , min_x: [-0.55164297 -0.55164297] , f(min_x): 1.0476880432938573 , g1: -0.39138007178622225 , g2: -6.0680726476314994

№ 7 , c

## Оптимизация функции с методом скользящего градиента

In [57]:
def grad_g1(x: np.array) -> np.array:
    return np.array([
        2 * x[0],
        2 * x[1]
    ])


def grad_g2(x: np.array, alpha: float) -> np.array:
    return np.array([
        1,
        alpha
    ])


def sliding_gradient_descent(
        func: TargetFunction, 
        x_0: np.array, 
        r: float,
        alpha: float,
        step_size: float = 0.01, 
        stop_factor: float = 0.001,
        close_area_threshold: float = 0.01,
        max_iterations: int = 1000
) -> (np.array, float):
    for iteration in range(1, max_iterations + 1):
        # Делаем шаг
        gradient = func.calculate_gradient(x_0)
        x_1 =  x_0  - step_size * gradient
        
        # Проверяем граничные условия ->
        g1_x1 = x_1[0] ** 2 + x_1[1] ** 2 - r ** 2
        g1_x0 = x_0[0] ** 2 + x_0[1] ** 2 - r ** 2
        g2_x1 = x_1[0] + alpha * x_1[1]
        g2_x0 = x_0[0] + alpha * x_0[1]
        
        # Выводим историю
        history = {
            'x_0': x_0,
            'x_1': x_1,
            'function_value': func.calculate(x_1),
            'gradient_norm': np.linalg.norm(gradient),
            'g1_x0': g1_x0,
            'g1_x1': g1_x1,
            'g2_x0': g2_x0,
            'g2_x1': g2_x1
        } 
        print(iteration, history)
        
        # Если были в допустимой области и вышли по g1
        if g1_x0 <= 0 < g1_x1:
            print('были в допустимой области и вышли по g1')
            x_1 = x_0 - gradient * (x_0[0] ** 2 + x_0[1] ** 2 - r ** 2) / np.dot(grad_g1(x_0), gradient)
            
        # Если были в допустимой области и вышли по g2
        elif g2_x0 <= 0 < g2_x1:
            print('были в допустимой области и вышли по g2')
            x_1 = x_0 - gradient * (x_0[0] + alpha * x_0[1]) / np.dot(grad_g2(x_0, alpha), gradient)
            
        # Eсли уже были на границе g1 и делаем шаг
        elif np.linalg.norm(g1_x0) < close_area_threshold and 0 < g1_x1:
            print('уже были на границе g1 и делаем шаг')
            gradient_g1 = grad_g1(x_0)
            x_1 = x_0 - step_size * (gradient - np.dot(gradient, gradient_g1) / np.linalg.norm(gradient_g1))
            # Доводка
            gradient_g1 = grad_g1(x_1)
            g1_x1 = x_1[0] ** 2 + x_1[1] ** 2 - r ** 2
            x_1 = x_1 - gradient_g1 * g1_x1 / np.dot(gradient_g1, gradient_g1)
            
        # Eсли уже были на границе g2 и делаем шаг
        elif np.linalg.norm(g2_x0) < close_area_threshold and 0 < g2_x1:
            'уже были на границе g2 и делаем шаг'
            gradient_g2 = grad_g2(x_0, alpha)
            x_1 = x_0 - step_size * (gradient - np.dot(gradient, gradient_g2) / np.linalg.norm(gradient_g2))
            # Доводка
            gradient_g2 = grad_g2(x_1, alpha)
            g2_x1 = x_1[0] + alpha * x_1[1]
            x_1 = x_1 - gradient_g2 * g2_x1 / np.dot(gradient_g2, gradient_g2)
        # <- Проеряем граничные условия
        
        # Если норма градиента меньше порога - выходим
        if np.linalg.norm(gradient) < stop_factor:
            return x_1, func.calculate(x_1)
        # Иначе - следующий шаг
        else:
            x_0 = x_1

    return x_1, func.calculate(x_1, add_calculations_count=False)

In [54]:
my_func = TargetFunction('Gregory', 'Matsnev')

In [61]:
min_x, min_value = sliding_gradient_descent(my_func, np.array([0, 0]), r=1, alpha=14)

print('\n' ,f'Minimizer: {min_x}', f'Value: {min_value}')

g1 = min_x[0] ** 2 + min_x[1] ** 2 - 1
g2 = min_x[0] + 14 * min_x[1]

print(g1, g2)

# my_func.flush_calculations_count()
# my_func.flush_derivative_calculations_count()

1 {'x_0': array([0, 0]), 'x_1': array([-0.01, -0.01]), 'function_value': 1.9800490002000837, 'gradient_norm': 1.4142135623730951, 'g1_x0': -1, 'g1_x1': -0.9998, 'g2_x0': 0, 'g2_x1': -0.15000000000000002}
2 {'x_0': array([-0.01, -0.01]), 'x_1': array([-0.01993, -0.01993]), 'function_value': 1.9603346346899402, 'gradient_norm': 1.40431398659041, 'g1_x0': -0.9998, 'g1_x1': -0.9992055902455734, 'g2_x0': -0.15000000000000002, 'g2_x1': -0.298949991424979}
3 {'x_0': array([-0.01993, -0.01993]), 'x_1': array([-0.02979048, -0.02979048]), 'function_value': 1.9408539077109221, 'gradient_norm': 1.3944832295244873, 'g1_x0': -0.9992055902455734, 'g1_x1': -0.9982250540180344, 'g2_x0': -0.298949991424979, 'g2_x1': -0.4468572736021313}
4 {'x_0': array([-0.02979048, -0.02979048]), 'x_1': array([-0.03958194, -0.03958194]), 'function_value': 1.9216038738656471, 'gradient_norm': 1.3847203494536813, 'g1_x0': -0.9982250540180344, 'g1_x1': -0.9968665406219273, 'g2_x0': -0.4468572736021313, 'g2_x1': -0.5937290